In [14]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

In [15]:
# 1. RunnableSequence - Basic Chain using | operator
# Automatically created when chaining with |

from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")

# This creates a RunnableSequence automatically
chain = prompt | model | StrOutputParser()

# Invoke the chain
result = chain.invoke({"topic": "programming"})
print(result)

Why do programmers prefer dark mode?

Because light attracts bugs!


In [16]:
from langchain_core.runnables import RunnableSequence

prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")

chain = RunnableSequence(prompt, model, StrOutputParser())
result = chain.invoke({"topic": "artificial intelligence"})
print(result)


Why did the AI break up with the calculator?

Because it felt their relationship was too **calculating** and lacked **emotional intelligence**!


In [17]:
# 2. RunnableParallel - Run multiple chains simultaneously
# Returns a dictionary with results from each branch

from langchain_core.runnables import RunnableParallel

joke_prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
poem_prompt = ChatPromptTemplate.from_template("Write a 2-line poem about {topic}")

joke_chain = joke_prompt | model | StrOutputParser()
poem_chain = poem_prompt | model | StrOutputParser()

# Run both chains in parallel
parallel_chain = RunnableParallel(
    joke=joke_chain,
    poem=poem_chain
)

result = parallel_chain.invoke({"topic": "cats"})
print("Joke:", result["joke"])
print("\nPoem:", result["poem"])

Joke: Why was the cat sitting on the computer?

Because it wanted to keep an eye on the **mouse**!

Poem: Soft paws tread silent, a shadow in the night,
Then purring warmth, a comforting, gentle light.


In [18]:
# 3. RunnablePassthrough - Pass input through unchanged
# Useful for including original input alongside transformed data

from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    "Answer the question based on context.\nContext: {context}\nQuestion: {question}"
)

# Passthrough keeps the original question while adding context
chain = (
    {
        "context": lambda x: "Python is a programming language created by Guido van Rossum.",
        "question": RunnablePassthrough()  # Passes the input unchanged
    }
    | prompt
    | model
    | StrOutputParser()
)

result = chain.invoke("Who created Python?")
print(result)

Guido van Rossum


In [19]:
# 4. RunnableLambda - Wrap any Python function as a Runnable
# Makes custom functions chainable

from langchain_core.runnables import RunnableLambda

# Custom function to process text
def word_count(text: str) -> dict:
    return {
        "text": text,
        "word_count": len(text.split()),
        "char_count": len(text)
    }

# Wrap function as Runnable
word_counter = RunnableLambda(word_count)

# Use in a chain
prompt = ChatPromptTemplate.from_template("Write a short sentence about {topic}")
chain = prompt | model | StrOutputParser() | word_counter

result = chain.invoke({"topic": "AI"})
print(f"Text: {result['text']}")
print(f"Word Count: {result['word_count']}")
print(f"Character Count: {result['char_count']}")

Text: AI is rapidly transforming how we live and work.
Word Count: 9
Character Count: 48


In [20]:
# 5. RunnableBranch - Conditional routing based on input
# Like if-else for chains

from langchain_core.runnables import RunnableBranch

# Define different prompts for different topics
positive_prompt = ChatPromptTemplate.from_template(
    "Respond enthusiastically to: {input}"
)
negative_prompt = ChatPromptTemplate.from_template(
    "Respond empathetically to: {input}"
)
default_prompt = ChatPromptTemplate.from_template(
    "Respond neutrally to: {input}"
)

# Create conditional branch
branch = RunnableBranch(
    # (condition, runnable) pairs
    (lambda x: "happy" in x["input"].lower(), positive_prompt | model | StrOutputParser()),
    (lambda x: "sad" in x["input"].lower(), negative_prompt | model | StrOutputParser()),
    # Default fallback
    default_prompt | model | StrOutputParser()
)

# Test with different inputs
print("Happy input:", branch.invoke({"input": "I'm so happy today!"}))
print("\nSad input:", branch.invoke({"input": "I feel sad"}))

Happy input: **YES! That's FANTASTIC news!** 🎉

Tell me EVERYTHING! What's making you so incredibly happy today? I'm practically vibrating with excitement to hear about it! Spill the beans! 🤩✨

Sad input: "I'm so sorry to hear you're feeling sad. That sounds really tough, and I want you to know you're not alone in feeling this way. It takes a lot of strength to even say that you're feeling sad, and I appreciate you sharing that with me.

Is there anything at all you'd like to talk about, or would you prefer just to sit with it for a bit? No pressure either way. I'm here to listen if you want to share, or just to be present if that's what you need.

Whatever you're going through, please be gentle with yourself. It's okay to feel sad, and it's okay to not have all the answers right now."


## Runnable Methods Summary

All Runnables share these common methods:

| Method | Description |
|--------|-------------|
| `invoke(input)` | Run with single input, return single output |
| `batch([inputs])` | Run with list of inputs, return list of outputs |
| `stream(input)` | Stream output chunks as they're generated |
| `ainvoke(input)` | Async version of invoke |
| `abatch([inputs])` | Async version of batch |
| `astream(input)` | Async version of stream |
| `bind(**kwargs)` | Bind parameters to the runnable |
| `with_fallbacks([runnables])` | Add fallback runnables |
| `with_retry()` | Add retry logic |
| `pick(keys)` | Select specific keys from output |
| `assign(**kwargs)` | Add new keys to output |

In [21]:
# 9. Batch Processing - Process multiple inputs efficiently

prompt = ChatPromptTemplate.from_template("What is the capital of {country}?")
chain = prompt | model | StrOutputParser()

# Process multiple inputs at once
countries = [
    {"country": "France"},
    {"country": "Japan"},
    {"country": "Brazil"}
]

results = chain.batch(countries)
for country, result in zip(countries, results):
    print(f"{country['country']}: {result}")

France: The capital of France is **Paris**.
Japan: The capital of Japan is **Tokyo**.
Brazil: The capital of Brazil is **Brasília**.


In [23]:
# 10. Streaming - Get output as it's generated

prompt = ChatPromptTemplate.from_template("Write a short story about {topic}")
chain = prompt | model | StrOutputParser()

# Stream the response
print("Streaming response:")
for chunk in chain.stream({"topic": "a robot learning to paint"}):
    print(chunk, end="", flush=True)

Streaming response:
Unit 734, designated "Artificer" by its creators, hummed with a quiet efficiency. Its metallic fingers, designed for intricate circuit board assembly, were now poised over a pristine canvas. Its optical sensors, usually scanning for microscopic defects, were focused on a still life arranged by its programmer: a ceramic apple, a glass of water, and a single, wilting rose.

Artificer had been programmed with an extensive database of art history, color theory, and brushstroke techniques. It could analyze a Rembrandt with the precision of a laser scanner and replicate a Monet with algorithmic accuracy. But understanding and *feeling* were two vastly different subroutines.

Its initial attempts were… sterile. It mixed pigments with perfect, calculated ratios. It applied paint with unwavering strokes, each line mathematically precise. The apple was rendered with flawless shading, the water captured with photographic realism, the rose’s petals arranged in an anatomically c